Dependencies:  
OpenCV (installation: $ sudo pip install opencv-contrib-python)  
TA-Lib installation and documentation
https://mrjbq7.github.io/ta-lib/install.html  
(for Windows machines look: https://forum.catalystcrypto.io/t/installing-ta-lib-on-windows/63)

no meu caso:
1. Fiz o Download da versão cp35-win_amd64 (superiores não funcionaram)
2. Abri o prompt de command via Visual Studio>Ferramentas>VS Command promprt
3. Nesse terminal:

In [1]:
import talib
import numpy as np
import pandas as pd
import math
import cv2 as cv
import os
#from sklearn import preprocessing
MINTIMEPERIOD=6
MAXTIMEPERIOD=20

In [13]:
def myCMFI(high, low, close, volume, timeperiod=2):
    clv = ((close-low)-(high-close))/(high-low)
    sum_mfv = talib.SUM((clv*volume), timeperiod=timeperiod)
    sum_volume = talib.SUM(volume, timeperiod=timeperiod)
    return (sum_mfv/sum_volume)

In [14]:
def myHMA(close, timeperiod=4):
    wma = talib.WMA(close, timeperiod=timeperiod)
    wma2 = 2*(talib.WMA(close, timeperiod=timeperiod//2))
    newPeriod= int(math.sqrt(timeperiod))
    return talib.WMA( wma2 - wma,timeperiod=newPeriod)


In [15]:
def Mat2Image(matrix, fileName):
    #preprocessing.normalize(matrix)
    minimun = np.amin(np.min(matrix))
    maximun = np.amax(np.amax(matrix))
    diff = maximun-minimun
    #print("max= %.1f, min= %.1f"%(minimun,maximun))
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            matrix[i,j] = 255*((matrix[i,j]-minimun)/(diff))
    cv.imwrite(fileName+'.png', matrix)


In [177]:
symbol = "DIS"
file = "labels/"+symbol+".csv"
table = pd.read_csv(file, index_col='Date')

adj_close = table['Adj Close'].values
close = table['Close'].values
high = table['High'].values
low = table['Low'].values
volume = table['Volume'].values

os.makedirs("./images",exist_ok=True)
os.makedirs("./images/buy",exist_ok=True)
os.makedirs("./images/hold",exist_ok=True)
os.makedirs("./images/sell",exist_ok=True)

In [178]:
table.head()

,Open,High,Low,Close,Adj Close,Volume,Labels
Date,,,,,,,
2001-11-01,18.382643,18.826429,18.205128,18.570021,14.831716,5533300,NaN
2001-11-02,18.786982,18.836292,18.441814,18.609467,14.863221,6932700,NaN
2001-11-05,18.984221,19.230770,18.737673,18.895464,15.091644,7300100,NaN
2001-11-06,18.392506,18.491125,18.096647,18.491125,14.768703,11329000,NaN
2001-11-07,18.244576,18.323471,18.027613,18.205128,14.540280,9951200,NaN


In [179]:
df_SMA = pd.DataFrame(index=table.index)
df_EMA = pd.DataFrame(index=table.index)
df_WMA = pd.DataFrame(index=table.index)
df_HMA = pd.DataFrame(index=table.index)
df_TEMA = pd.DataFrame(index=table.index)
df_CCI = pd.DataFrame(index=table.index)
df_CMO = pd.DataFrame(index=table.index)
df_MACD = pd.DataFrame(index=table.index)
df_PPO = pd.DataFrame(index=table.index)
df_ROC = pd.DataFrame(index=table.index)
df_CMFI = pd.DataFrame(index=table.index)
df_DX = pd.DataFrame(index=table.index)
df_SAR = pd.DataFrame(index=table.index)
df_RSI = pd.DataFrame(index=table.index)
df_WILLR = pd.DataFrame(index=table.index)

In [180]:
for i in range(MINTIMEPERIOD,MAXTIMEPERIOD+1):
    df_RSI[str(i)] = talib.RSI(close, timeperiod=i)
    df_WILLR[str(i)] = talib.WILLR(high, low, close,timeperiod=i)
    df_WMA[str(i)] = talib.WMA(close, timeperiod=i)
    df_EMA[str(i)] = talib.EMA(close, timeperiod=i)
    df_SMA[str(i)] = talib.SMA(close, timeperiod=i)
    df_HMA[str(i)] = myHMA(close, timeperiod=i)
    df_TEMA[str(i)] = talib.TEMA(close, timeperiod=i)
    df_CCI[str(i)] = talib.CCI(high, low, close, timeperiod=i)
    df_CMO[str(i)] = talib.CMO(close,timeperiod=i)
    df_MACD[str(i)] = talib.MACD(close,fastperiod=i,slowperiod=i*2)[1] #macdsignal
    df_PPO[str(i)] = talib.PPO(close, fastperiod=i,slowperiod=i*2)
    df_ROC[str(i)] = talib.ROC(close,timeperiod=i)
    df_CMFI[str(i)] = myCMFI(high, low, close, (volume)*1.0, timeperiod=i)
    df_DX[str(i)] = talib.DX(high, low, close,timeperiod=i)
    df_SAR[str(i)] = talib.SAR(high, low)
    

In [181]:
for index in table.index: 
    img = np.zeros([15,15])
    
    img[0] = df_RSI.loc[index].values
    img[1] = df_WILLR.loc[index].values
    img[2] = df_WMA.loc[index].values
    img[3] = df_EMA.loc[index].values
    img[4] = df_SMA.loc[index].values
    img[5] = df_HMA.loc[index].values
    img[6] = df_TEMA.loc[index].values
    img[7] = df_CCI.loc[index].values
    img[8] = df_CMO.loc[index].values
    img[9] = df_MACD.loc[index].values
    img[10] = df_PPO.loc[index].values
    img[11] = df_ROC.loc[index].values
    img[12] = df_CMFI.loc[index].values
    img[13] = df_DX.loc[index].values
    img[14] = df_SAR.loc[index].values
    
    if np.argwhere(np.isnan(img)).size != 0: #there is some NaN
        continue
    else:
        label = table.loc[index]['Labels']
        filename = './images/'+str(label)+'/'+symbol+'_'+index
        Mat2Image(img,filename)
    

In [33]:
type(index)

str

In [87]:
def myNormalize(array):
    minimum = np.amin(array)
    maximum = np.amax(array)
    diff = maximum - minimum
    if diff == 0: 
        diff = 0.001
    t_m = np.ones(len(array))*minimum
    t = array - t_m
    return t/diff

In [ ]:
numH =np.array([33.5,34.6,34.9,35.0,34.6])
numL =np.array([32.7,33.5,34.3,34.1,34.0])
numC =np.array([33.4,34.4,34.4,34.2,34.6])
volume = np.array([481244000.0,67590800.0,52731400.0,47736400.0,76304000.0])
print(myDMI(numH, numL, numC, volume))

In [56]:
for i in range(10):
    if i%2 == 0:
        continue
    print(i)

1
3
5
7
9


In [92]:
myNormalize(x)

6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
Name: 2003-05-12, dtype: float64

In [125]:
table.index

Index(['2002-01-02', '2002-01-03', '2002-01-04', '2002-01-07', '2002-01-08',
       '2002-01-09', '2002-01-10', '2002-01-11', '2002-01-14', '2002-01-15',
       ...
       '2006-12-15', '2006-12-18', '2006-12-19', '2006-12-20', '2006-12-21',
       '2006-12-22', '2006-12-26', '2006-12-27', '2006-12-28', '2006-12-29'],
      dtype='object', name='Date', length=1259)

In [93]:
x

6     24.92855
7     24.92855
8     24.92855
9     24.92855
10    24.92855
11    24.92855
12    24.92855
13    24.92855
14    24.92855
15    24.92855
16    24.92855
17    24.92855
18    24.92855
19    24.92855
20    24.92855
Name: 2003-05-12, dtype: float64

In [88]:
x = df_SAR.loc['2003-05-12']

In [ ]:
print(df_SAR.head())

In [ ]:
df_SAR.iloc[0]

In [ ]:
teste = np.ones([2,15])
new = df_SAR.loc['2002-01-02'].values
#teste[0] = new+2
#teste[1] = new
teste[1,3]=np.nan
print(teste)

In [ ]:
t = np.argwhere(np.isnan(teste))

In [9]:
((file.split("/"))[-1].split("."))[0]

'MSTF'

In [ ]:
#t=np.array([])
if (t.size != 0): print('tem NaN')

In [ ]:
df_SAR.iat[1,0]

In [ ]:
df_SAR.ix[1]

In [28]:
for i in range(MINTIMEPERIOD,MAXTIMEPERIOD+1): print(i)

6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [7]:
import os
folds = os.listdir("./experimento")
#os.getcwd()

In [8]:
folds

['fold_1',
 'fold_10',
 'fold_2',
 'fold_3',
 'fold_4',
 'fold_5',
 'fold_6',
 'fold_7',
 'fold_8',
 'fold_9']

In [9]:
from subprocess import Popen as popen
import os
folds = os.listdir("./experimento")
os.makedirs("./experiment2", exist_ok=True)
labels = ["buy", "hold", "sell"]
directories = ["train", "validation"]

for fold in folds:
    print(os.getcwd())
    for dirr in directories:
        for label in labels:
            sourcePath = "./experimento/"+fold+"/dataset/"+dirr+"/"+label+"/"
            destPath = "./experiment2/"+fold+"/dataset/"+dirr+"/"+label+"/"
            os.makedirs(destPath, exist_ok=True)
            imgs = os.listdir(sourcePath)
            for img in imgs:
                popen(['ln','-s', '../../../../../rps/'+label+'/'+img, img], cwd=os.path.abspath(destPath))
                

D:\agnal\unicamp\projetos\indicators
D:\agnal\unicamp\projetos\indicators
D:\agnal\unicamp\projetos\indicators
D:\agnal\unicamp\projetos\indicators
D:\agnal\unicamp\projetos\indicators
D:\agnal\unicamp\projetos\indicators
D:\agnal\unicamp\projetos\indicators
D:\agnal\unicamp\projetos\indicators
D:\agnal\unicamp\projetos\indicators
D:\agnal\unicamp\projetos\indicators
